# Singularity Extractor - Non-optimized

In [42]:
import numpy as np
import tensorflow as tf

seed = 173
np.random.seed(seed)

In [60]:
kernel_size = 5
strides = 2
padding = "CONSTANT"
input_mat = tf.constant([
    [ 1,  2,  3,  4,  5, 6],
    [ 6,  7,  8,  9, 10, 11],
    [11, 12,  1, 14, 15, 16],
    [16, 17, 18, 19, 20, 21],
    [21, 22, 23, 24, 25, 26],
    [27, 28, 29, 30, 31, 32]
])
degree = 6
margin = 2

input_shape = input_mat.shape
output = np.zeros(input_shape)
radius = int((kernel_size - 1)/2) * strides
matpad = tf.pad(input_mat, tf.constant([[radius, radius], [radius, radius]]), mode=padding)

for y in range(input_shape[1]):
    for x in range(input_shape[0]):
        _x, _y = x + radius, y + radius
        kernel = matpad[_y - radius:_y + 1 + radius:strides, _x - radius: _x + 1 + radius:strides]
        redsum = tf.reduce_sum(kernel)
        output[y, x] = ((redsum - input_mat[x, y]) / redsum) ** degree

print(input_mat)
print("-----------------------")
print((tf.round(output * 100)/100)[margin:-margin, margin:-margin])

tf.Tensor(
[[ 1  2  3  4  5  6]
 [ 6  7  8  9 10 11]
 [11 12  1 14 15 16]
 [16 17 18 19 20 21]
 [21 22 23 24 25 26]
 [27 28 29 30 31 32]], shape=(6, 6), dtype=int32)
-----------------------
tf.Tensor(
[[0.94 0.4 ]
 [0.59 0.5 ]], shape=(2, 2), dtype=float64)


# Optimized

In [61]:
xx = tf.pad(tf.stack([input_mat, input_mat, input_mat]), [[0, 0],[radius, radius], [radius, radius]], mode="REFLECT")
xx.shape

TensorShape([3, 14, 14])

In [62]:
xx = tf.reshape(xx, (*xx.shape,1))

In [63]:
ones = tf.ones((3, 3, 1, 1), dtype="int32")

In [64]:
(xx.shape, ones.shape)

(TensorShape([3, 14, 14, 1]), TensorShape([3, 3, 1, 1]))

In [65]:
conv = tf.nn.convolution(xx, ones, strides=1, padding="SAME")
conv.shape

TensorShape([3, 14, 14, 1])

In [66]:
mrgn_0  =  int((conv.shape[0] - xx.shape[0])/2) + margin
mrgn_1  =  int((conv.shape[1] - xx.shape[1])/2) + margin
selection_conv = conv[:, mrgn_0:-mrgn_0, mrgn_1:-mrgn_1, :]
selection_input = xx[:, margin:-margin, margin:-margin, :]
((selection_conv - selection_input) / selection_conv) ** degree

<tf.Tensor: shape=(3, 10, 10, 1), dtype=float64, numpy=
array([[[[0.94420053],
         [0.44879532],
         [0.51479145],
         [0.44879532],
         [0.94420053],
         [0.45558655],
         [0.49327018],
         [0.47740225],
         [0.49327018],
         [0.45558655]],

        [[0.42375278],
         [0.41237754],
         [0.531441  ],
         [0.41237754],
         [0.42375278],
         [0.4323276 ],
         [0.49327018],
         [0.46987834],
         [0.49327018],
         [0.4323276 ]],

        [[0.72295859],
         [0.77463937],
         [0.87385822],
         [0.77463937],
         [0.72295859],
         [0.68720534],
         [0.66102922],
         [0.61862485],
         [0.66102922],
         [0.68720534]],

        [[0.42375278],
         [0.41237754],
         [0.531441  ],
         [0.41237754],
         [0.42375278],
         [0.4323276 ],
         [0.49327018],
         [0.46987834],
         [0.49327018],
         [0.4323276 ]],

        [[0.9442